In [4]:
import numpy as np

def create_matrix(kon, kappa):
    # Identity matrices
    I_kon = np.eye(kon)  # kon x kon identity matrix
    I_1 = np.eye(1)      # 1 x 1 identity matrix
    I_kappa_kon = np.eye(kappa - kon)  # (kappa-kon) x (kappa-kon) identity matrix

    # Zero matrices
    Z_11 = np.zeros((kon, 1))
    Z_13 = np.zeros((kon, kappa - kon))
    Z_22 = np.zeros((1, kon))
    Z_23 = np.zeros((1, kappa - kon))
    Z_31 = np.zeros((kappa - kon, 1))
    Z_32 = np.zeros((kappa - kon, kon))

    # Assemble the block matrix
    matrix = np.block([
        [Z_11, I_kon, Z_13],
        [I_1, Z_22, Z_23],
        [Z_31, Z_32, I_kappa_kon]
    ])

    return matrix

# Example usage
kon = 3
kappa = 7
result = create_matrix(kon, kappa)
print("Generated Matrix:")
print(result)


Generated Matrix:
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [5]:
import numpy as np
import redis
import pickle
import timeit
from scipy.sparse import csr_matrix



rs = redis.StrictRedis(host='172.18.0.1',port=6379,db=2,password="chlpw1039") 
print(pickle.loads(rs.mget(f'M_in_CAV_0')[0]))
K = pickle.loads(rs.mget(f'K_in_CAV_0')[0])
Yif = pickle.loads(rs.mget(f'Yif_in_CAV_0')[0])
z_plus = pickle.loads(rs.mget(f'g_initial_in_CAV_0')[0])
# 提前将K转换为稀疏矩阵（只需一次）
K_sparse = csr_matrix(K)

# 密集矩阵版本
def dense_version():
    return K @ Yif @ z_plus

# 稀疏优化版本
def sparse_version():
    temp = K_sparse.dot(Yif)
    return temp.dot(z_plus)

# 测试运行时间
t_dense = timeit.timeit(dense_version, number=1000)
t_sparse = timeit.timeit(sparse_version, number=1000)
print(f"密集版本: {t_dense:.4f}s，稀疏版本: {t_sparse:.4f}s，加速比: {t_dense/t_sparse:.1f}x")


[[0.5 0.  0.  ... 0.  0.  0. ]
 [0.  0.5 0.  ... 0.  0.  0. ]
 [0.  0.  0.5 ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 1.6 0.  0. ]
 [0.  0.  0.  ... 0.  1.6 0. ]
 [0.  0.  0.  ... 0.  0.  1. ]]


In [6]:
import numpy as np

s = np.array([1,2,3,4,0])
s_plus = s.clip(min=1,max=3)
print(s_plus)

[1 2 3 3 1]
